### 加入LDA建立的2個新主題特徵
* 餐廳飲食、環境設備

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
By_sentence_pos=pd.read_csv('By_sentence_pos.csv')
By_sentence_pos['pos']=By_sentence_pos['pos'].apply(lambda x:eval(x))
By_sentence_pos['token_ckip']=By_sentence_pos['pos'].apply(lambda x:[i[0].upper().strip() for i in x])
By_sentence_pos['Counter_pos']=By_sentence_pos['pos'].apply(Counter)

In [ ]:
##計算總辭頻
Counter_pos=By_sentence_pos[['Counter_pos']].copy()
Counter_pos['group']=Counter_pos.index%2000
group_Counter=Counter_pos.groupby('group')['Counter_pos'].sum()
total_Counter=group_Counter.sum()
ckip_pos=['VA','VB','VC','VD','VE','VH','VHC','VK','VJ','VL','VCL','Na','Nc','Nv']  #刪Nb
total_word_counter={}
for tuple_,count in total_Counter.most_common():
    word,pos=tuple_
    count_=0
    word=word.upper().strip()
    if count>1:
        count_=total_word_counter.get(word,0)
        if pos in ckip_pos:
            count+=count_
            total_word_counter[word]=count
        elif word =='101':
            count+=count_
            total_word_counter[word]=count
        elif pos=='FW':
            if len(word)>=3:
                count+=count_
                total_word_counter[word]=count
total_word_counter=Counter(total_word_counter)
total_word_counter = {k : v for k,v in total_word_counter.items() if v >1}

In [ ]:
#原先分類字典(加入LDA主題的)
classif = pd.read_excel('./feature_dict_lda.xlsx')
classif.columns=['service','place','clean','cp','comfort','sleep','environment','food']
classif=classif.to_dict()
service_list=np.array(list(classif['service'].values()))
classif['service']=service_list[service_list!='nan']
place_list=np.array(list(classif['place'].values()))
classif['place']=place_list[place_list!='nan']
clean_list=np.array(list(classif['clean'].values()))
classif['clean']=clean_list[clean_list!='nan']
cp_list=np.array(list(classif['cp'].values()))
classif['cp']=cp_list[cp_list!='nan']
comfort_list=np.array(list(classif['comfort'].values()))
classif['comfort']=comfort_list[comfort_list!='nan']
sleep_list=np.array(list(classif['sleep'].values()))
classif['sleep']=sleep_list[sleep_list!='nan']
food_list=np.array(list(classif['food'].values()))
classif['food']=food_list[food_list!='nan']
environment_list=np.array(list(classif['environment'].values()))
classif['environment']=environment_list[environment_list!='nan']

In [ ]:
By_sentence_pos['token_ckip']=By_sentence_pos['pos'].apply(lambda x:[i[0].upper().strip() for i in x if i[0] in total_word_counter.keys()])
cut_word_sample=By_sentence_pos[['token_ckip']].copy()
word_list=list(total_word_counter.keys())
cut_word_sample=cut_word_sample.reset_index(drop=True)
cut_word_sample['Counter']=cut_word_sample['token_ckip'].apply(Counter)
cut_word_sample=cut_word_sample[cut_word_sample.Counter!={}]

In [ ]:
##利用卡方分配進行分類
from collections import defaultdict
from datetime import datetime
def find_category(classif,x):
    dict_=defaultdict(list)
    for cate in classif.keys():
        count_=0
        same=set(x.keys()).intersection(set(classif[cate]))
        for sa in same:
            count_+=x[sa]
        dict_[count_].append(cate)
    category=dict_[max(dict_.keys())]
    if max(dict_.keys())==0:
        category= 'None'
    return(category)

new_flag=True
iteration=1
P=30
while( (new_flag==True)|(iteration>15)):
    print(iteration,datetime.now())
    C=cut_word_sample.shape[0]
    cut_word_sample['category']=cut_word_sample['Counter'].apply(lambda x: find_category(classif,x))
    for cate_ in classif.keys():
        cut_word_sample[f'Is_{cate_}']=cut_word_sample['category'].apply(lambda x: True if cate_ in x else False)
    output_list=[]
    for word_ in word_list:
        cut_word_sample['Is_Exist']=cut_word_sample['Counter'].apply(lambda x:True if word_ in x.keys() else False)
        for cate_ in classif.keys():
            topic_df=cut_word_sample[cut_word_sample[f'Is_{cate_}']]
            C1=topic_df['Counter'].apply(lambda x:x[word_]).sum()
            C2=total_word_counter[word_]-C1
            C3=topic_df[topic_df.Is_Exist==False].shape[0]
            C4=cut_word_sample[(cut_word_sample.Is_Exist==False)&(cut_word_sample.Is_service==False)].shape[0]
            try:
                Chi = (C*((C1*C4-C2*C3)**2))/((C1+C3)*(C2+C4)*(C1+C2)*(C3+C4))
            except ZeroDivisionError:
                Chi =0
            output_list.append([word_,cate_,Chi])
    total_num=pd.DataFrame(output_list,columns=['word','category','Chi'])
    new_flag=False
    for cate in classif.keys():
        new_flag=False
        keyword_list=list(classif[cate])
        new_word=total_num[total_num.category==cate].sort_values('Chi',ascending=False)[:P].word.values
        for new in new_word:
            if new not in keyword_list:
                keyword_list.append(new)
                new_flag=True
        classif[cate]=keyword_list
    new_classif=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classif.items() ]))
    new_classif.to_xlsx('./feature_8topic.xlsx',index=False)
    iteration+=1